# Basic Capacioty Expansion Practical

In this practical, we aim to build a simple capacity expansion model for hydrogen supoply chain. The code is developed in python, and the demand, fuel price, network data, and zone characterisitcs are given.

### 1. Load and Process the data

Let's start by loading the data we have. Load the data from 'Ex03' Folder.

In [69]:
# Import the packages first

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [70]:
# we save the adress to the data directory in a variable, so we can use it later
data_directory = "/workspaces/Supply_Chain_Analytics_2026/Exercise_Files/Ex03"

Fuels = pd.read_csv(f"{data_directory}/Fuels_data.csv")
Gen_data = pd.read_csv(f"{data_directory}/HSC_Gen_Data.csv")
Load = pd.read_csv(f"{data_directory}/HSC_load.csv")
Network = pd.read_csv(f"{data_directory}/HSC_Pipelines.csv")
Zone_data = pd.read_csv(f"{data_directory}/Zone_Char.csv")


In [71]:
# let's have a look at the data -- Remove the comment tags to show the data
Fuels.head()
#Gen_data.head()
#Load.head()
#Network.head()
#Zone_data.head()

# you can also use .describe() to get a statistical summary of the data
Load.describe()

,Time_Index,Load_HSC_Tonne_z1,Load_HSC_Tonee_z2,Load_HSC_Tonne_z3,Load_HSC_Tonne_z4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,4380.500000,17.906473,8.635753,6.222432,12.485205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2528.938512,8.924145,4.265556,3.142557,4.797066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,4.200000,2.400000,1.400000,5.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2190.750000,11.200000,5.300000,3.800000,8.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4380.500000,17.000000,8.200000,5.900000,11.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,6570.250000,23.500000,11.400000,8.200000,15.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,8760.000000,58.000000,26.700000,20.300000,32.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We need to define the set of generators, storage units, pipelines, and timesteps, to be able to define variables and constraints by them. 

In [72]:
# Let G represent set of all generators in the model.
# in the HSC_Gen_Data, the column 'H_Gen_type' indicates whether a resource is a generator (>0) or storage unit (0). 
# If it is 1, it is a generator that procudes emission suh SMR, if it is 2, it is a generator that produces zero-emission hydrogen via electrolysis.
dfGen = Gen_data[Gen_data['H_Gen_type']>0]
G = dfGen.r_id
#print(G['Resource'])

# Let S represent set of all storage units in the model.
dfStorage = Gen_data[Gen_data['H_Gen_type']==0]
S = dfStorage.r_id
#print(S['Resource'])

# Let I represent set of all pipelines in the model.
I = Network['HSC_Pipelines']
#print(I)

# Let T represent set of all timesteps in the model
T = Load['Time_Index']

# Set of all zones is represented by Z
Z = Zone_data['Zones']


### 2. Create the Model

In [73]:
hsc = gp.Model("HSC_Capacity_Expansion")

### 3. Define the Variables

The variables in the model are either related to Capacity, Operation, or Policy. It is common in programming that the variables are named with "camelCase" for better readability. In this way, we start a variable name with lowercase 'v', and each new word starts with capital letter.

In [74]:
######## .......................#############
# Defining the Capacity decision variables #
######## .......................############

# New and Retired Generation Capacity variables
vNewGenCap = hsc.addVars(G, name="NewGenCap", lb=0, vtype=GRB.INTEGER)
vRetGenCap = hsc.addVars(G, name="RetGenCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Storage Capacity variables
vNewStorCap = hsc.addVars(S, name="NewStorCap", lb=0, vtype=GRB.INTEGER)
vRetStorCap = hsc.addVars(S, name="RetStorCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Pipeline Capacity variables
vNewPipe = hsc.addVars(I, name="NewPipeCap", lb=0, vtype=GRB.INTEGER)
vRetPipe = hsc.addVars(I, name="RetPipeCap", lb=0, vtype=GRB.INTEGER)


In [75]:
######## .......................#############
# Defining the Operation decision variables #
######## .......................############

# Generation from generators for each generator and timestep
vGen = hsc.addVars(G, T, name="Gen", lb=0, vtype=GRB.CONTINUOUS) 

# Storage charge and discharge for each storage unit and timestep
vStorCharge = hsc.addVars(S, T, name="StorCharge", lb=0, vtype=GRB.CONTINUOUS) 
vStorDischarge = hsc.addVars(S, T, name="StorDischarge", lb=0, vtype=GRB.CONTINUOUS)
# State of Charge for each storage unit and timestep
vStorSOC = hsc.addVars(S, T, name="StorSOC", lb=0, vtype=GRB.CONTINUOUS)

# Pipeline flow for each pipeline and timestep
vPipeFlow = hsc.addVars(I, T, name="PipeFlow", vtype=GRB.CONTINUOUS) # Note: can be negative for bi-directional flow

In [76]:
######## .......................#############
# Defining the Policy decision variables   #
######## .......................############

# In this simplified example, the only policy variable is the non-served hydrogen demand. With this variabel, the model can choose to no serve a part of the demand if it is too constly to serve it.
# For example, if the cost of electricity is too high and no other generation options are available, the model can choose to not serve a part of the hydrogen demand and pay a penalty, instead of building expensive new capacity.

vNSD = hsc.addVars(Z, T, name="NonServedDemand", lb=0, vtype=GRB.CONTINUOUS)

# We consider a net-zero HSC in this example, so every unit of CO2 emitted must be have a penalty. So, we do not need an emission variable, we just need to calculate the rmission generated from SMRs.

### 3. Objective

As previously mentioned in the lecture slides, the objective of the model is comprised of cost of investment for new capacity, cost of operation, and cost of penalty terms in the system.

In [77]:
# For every resource, total capacity is equal to existing capacity plus new capacity minus retired capacity.
total_gen_cap = {}
for g in G:
    # Get the existing capacity for generator g from Gen_data
    existing_cap = Gen_data[Gen_data['r_id'] == g]['Existing_cap_tonne_p_hr'].values[0]
    total_gen_cap[g] = existing_cap + vNewGenCap[g] - vRetGenCap[g]

total_sto_cap = {}
for s in S:
    existing_cap = Gen_data[Gen_data['r_id'] == s]['Existing_cap_tonne'].values[0]
    total_sto_cap[s] = existing_cap + vNewStorCap[s] - vRetStorCap[s]


total_pipe_cap = {}
for i in I:
    existing_cap = Network[Network['HSC_Pipelines'] == i]['Existing_Num_Pipes'].values[0]
    total_pipe_cap[i] = existing_cap + vNewPipe[i] - vRetPipe[i]


# Let's define the investment costs
gen_investment_cost = gp.quicksum(
    vNewGenCap[g] * Gen_data[Gen_data['r_id']== g]['Inv_cost_tonne_hr_p_yr'] 
    for g in G  
)
storage_investment_cost = gp.quicksum(
    vNewStorCap[s] * Gen_data[Gen_data['r_id']== s]['Inv_cost_tonne_p_yr'] 
    for s in S)

pipeline_investment_cost = gp.quicksum(
    vNewPipe[i] * Network[Network['HSC_Pipelines']== i]['Investment_cost_per_capacity']
    for i in I) 

total_investment_cost = gen_investment_cost + storage_investment_cost + pipeline_investment_cost


/tmp/ipykernel_2802/3302763958.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vNewGenCap[g] * Gen_data[Gen_data['r_id']== g]['Inv_cost_tonne_hr_p_yr']
/tmp/ipykernel_2802/3302763958.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vNewStorCap[s] * Gen_data[Gen_data['r_id']== s]['Inv_cost_tonne_p_yr']
/tmp/ipykernel_2802/3302763958.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vNewPipe[i] * Network[Network['HSC_Pipelines']== i]['Investment_cost_per_capacity']


In [78]:
# Fixed Operation and Maintenance Costs 
gen_fom_cost = gp.quicksum(
    total_gen_cap[g] * Gen_data[Gen_data['r_id']== g]['FOM_Cost_p_tonne_p_hr_yr'].values[0]
    for g in G  
) 

sto_fom_cost = gp.quicksum(
    total_sto_cap[s] * Gen_data[Gen_data['r_id']== s]['FOM_Cost_p_tonne_p_yr'].values[0]
    for s in S  
)

pipe_fom_cost = gp.quicksum(
    total_pipe_cap[i] * Network[Network['HSC_Pipelines']== i]['FOM_per_capacity'].values[0]
    for i in I  
)

# Variabnle Operation and Maintenance Costs -- We only consider fuel cost as variable O&M cost in this example

gen_vom_cost = gp.quicksum(
    vGen[g, t] * Fuels[Fuels['Time_Index'] == t][Gen_data[Gen_data['r_id']==g]['Fuel'].values[0]].values[0]
    for g in G for t in T
)

total_operation_cost = gen_fom_cost + sto_fom_cost + gen_vom_cost


In [81]:
# Total Penalty Costs

NSD_Cost = gp.quicksum(
    vNSD[z, t] * Zone_data[Zone_data['Zones']==z]['HSC_NSD_Cost'].values[0]
    for z in Z for t in T
)

#Emission_Cost = gp.quicksum(vGen[g,t] * Gen_data[Gen_data['r_id']==g]['Emission_per_tonne_H2'] * Zone_data[Zone_data['Zones']==Gen_data[Gen_data['Zone']==z]]['Emission_cost'] for g in G for t in T for z in Z if Gen_data[Gen_data['r_id']==g]['Zone'].values[0]==z )

Emission_Cost = gp.quicksum(
    vGen[g, t] *
    Gen_data.loc[Gen_data['r_id'] == g, 'Emission_per_tonne_H2'].iloc[0] *
    Zone_data.loc[
        Zone_data['Zones'] == Gen_data.loc[Gen_data['r_id'] == g, 'Zone'].iloc[0],
        'Emission_cost'
    ].iloc[0]
    for g in G for t in T
)


In [82]:
hsc.setObjective(total_investment_cost + total_operation_cost + NSD_Cost + Emission_Cost, GRB.MINIMIZE)